In [ ]:
from google.colab import drive 
drive.mount('/mntDrive')

Mounted at /mntDrive


In [ ]:
! rm -r ocrpostcorrection

In [ ]:
!git clone https://github.com/jvdzwaan/ocrpostcorrection.git

Cloning into 'ocrpostcorrection'...
remote: Enumerating objects: 723, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 723 (delta 88), reused 92 (delta 48), pack-reused 588
Receiving objects: 100% (723/723), 1.18 MiB | 2.66 MiB/s, done.
Resolving deltas: 100% (453/453), done.


In [ ]:
!pip install ./ocrpostcorrection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./ocrpostcorrection
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 451 kB 37.4 MB/s 
     |████████████████████████████████| 325 kB 67.9 MB/s 
     |████████████████████████████████| 58 kB 6.6 MB/s 
     |████████████████████████████████| 5.5 MB 42.8 MB/s 
     |████████████████████████████████| 182 kB 57.0 MB/s 
     |████████████████████████████████| 115 kB 14.1 MB/s 
     |████████████████████████████████| 212 kB 59.9 MB/s 
     |████████████████████████████████| 127 kB 34.9 MB/s 
     |██████

In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Loading data

In [ ]:
data_base_dir = Path('/Users/janneke/Documents/Documents – Janneke’s MacBook/data/ocrpostcorrection')

In [ ]:
data_base_dir = Path('/mntDrive/MyDrive/data/ocrpostcorrection')

### Dataset without duplicates

In [ ]:
in_file = data_base_dir/'icdar-task2-dataset-20221031'/'task2dataset-no-duplicates.csv'
data = pd.read_csv(in_file, index_col=0)
data = data.fillna('')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/janneke/Documents/Documents – Janneke’s MacBook/data/ocrpostcorrection/icdar-task2-dataset-20221031/task2dataset-no-duplicates.csv'

In [ ]:
train = data.query('dataset == "train"')
test = data.query('dataset == "test"')

### Test set from 'perfect' ICDAR output for error detection 

In [ ]:
# Local
test_data_dir = Path('/Users/janneke/Documents/Documents – Janneke’s MacBook/data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish')

In [ ]:
# Google Drive
test_data_dir = data_base_dir / 'ICDAR2019_POCR_competition_dataset' / 'ICDAR2019_POCR_competition_evaluation_4M_without_Finnish'

In [ ]:
from ocrpostcorrection.icdar_data import generate_data
from ocrpostcorrection.utils import create_perfect_icdar_output, icdar_output2simple_correction_dataset_df

data_test, md_test = generate_data(test_data_dir)

output = create_perfect_icdar_output(data_test)

test = icdar_output2simple_correction_dataset_df(output, data_test)

11it [00:06,  1.68it/s]


In [ ]:
from ocrpostcorrection.error_correction import generate_vocabs, get_text_transform

vocab_transform = generate_vocabs(train)
text_transform = get_text_transform(vocab_transform)

In [ ]:
from torch.utils.data import DataLoader

from ocrpostcorrection.error_correction import SimpleCorrectionDataset, collate_fn

max_len = 22
batch_size = 256

test_dataset = SimpleCorrectionDataset(test, max_len=max_len)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn(text_transform))

## Load model

In [ ]:
from ocrpostcorrection.error_correction import SimpleCorrectionSeq2seq

hidden_size = 256
dropout = 0.1
model = SimpleCorrectionSeq2seq(len(vocab_transform['ocr']), 
                                hidden_size, 
                                len(vocab_transform['gs']), 
                                dropout, 
                                max_len, 
                                teacher_forcing_ratio=0.5,
                                device=device)
model.to(device)    
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
!ls /mntDrive/MyDrive/data

ocrpostcorrection


In [ ]:
model_save_path = data_base_dir/'results'/'simple_correction_model'/'model.rar'

checkpoint = torch.load(model_save_path, map_location=torch.device(device))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                        
model = model.to(device)

In [ ]:
from ocrpostcorrection.error_correction import validate_model

validate_model(model, test_dataloader, device)

9.709021554397497

In [ ]:
from ocrpostcorrection.error_correction import predict_and_convert_to_str

predictions = predict_and_convert_to_str(model, test_dataloader, vocab_transform['gs'], device)

100%|██████████| 1014/1014 [05:14<00:00,  3.23it/s]


In [ ]:
test_results = test.query(f'len_ocr <= {max_len}').query(f'len_gs <= {max_len}').copy()

test_results['pred'] = predictions

In [ ]:
import edlib 

test_results['ed'] = test_results.apply(lambda row: edlib.align(row.ocr, row.gs)['editDistance'], axis=1)
test_results.ed.describe()

count    259466.000000
mean          2.985832
std           2.256691
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max          22.000000
Name: ed, dtype: float64

In [ ]:
from ocrpostcorrection.icdar_data import normalized_ed

test_results['ed_norm'] = test_results.apply(lambda row: normalized_ed(row.ed, row.ocr, row.gs), axis=1)
test_results.ed_norm.describe()

count    259466.000000
mean          0.364450
std           0.220131
min           0.045455
25%           0.200000
50%           0.333333
75%           0.500000
max           1.000000
Name: ed_norm, dtype: float64

In [ ]:
import edlib 

test_results['ed_pred'] = test_results.apply(lambda row: edlib.align(row.pred, row.gs)['editDistance'], axis=1)
test_results.ed_pred.describe()

count    259466.000000
mean          2.159296
std           2.555590
min           0.000000
25%           0.000000
50%           1.000000
75%           3.000000
max          23.000000
Name: ed_pred, dtype: float64

In [ ]:
import pandas as pd

pd.DataFrame([test_results.ed.describe(), test_results.ed_pred.describe()])

NameError: name 'test_results' is not defined

In [ ]:
test_results['ed_norm_pred'] = test_results.apply(lambda row: normalized_ed(row.ed_pred, row.pred, row.gs), axis=1)
test_results.ed_norm_pred.describe()

count    259466.000000
mean          0.262817
std           0.263147
min           0.000000
25%           0.000000
50%           0.200000
75%           0.400000
max           1.000000
Name: ed_norm_pred, dtype: float64

In [ ]:
(test_results.pred == test_results.gs).sum()/test_results.shape[0]

0.285467074684159

In [ ]:
test_results[test_results.pred == test_results.gs].sample(5)

,ocr,gs,ocr_aligned,gs_aligned,start,len_ocr,language,subset,dataset,len_gs,diff,pred,ed,ed_norm,ed_pred,ed_norm_pred
1774160,ladhend,lachend,ladhend,lachend,959,7,DE,DE3,test,7,0,lachend,1,0.142857,0,0.0
1634071,Zlügeln,Flügeln,Zlügeln,Flügeln,785,7,DE,DE3,test,7,0,Flügeln,1,0.142857,0,0.0
1745233,willenlog,willenlos,willenlog,willenlos,974,9,DE,DE3,test,9,0,willenlos,1,0.111111,0,0.0
1702184,ern{te,ernſte,ern{te,ernſte,989,6,DE,DE3,test,6,0,ernſte,1,0.166667,0,0.0
1795091,»Unbd,»Und,»Unbd,»Un@d,1210,5,DE,DE3,test,4,1,»Und,1,0.200000,0,0.0


In [ ]:
out_file = data_base_dir/'results'/'simple_correction_model'/'predictions.csv'
test_results.to_csv(out_file)

In [ ]:
out_file = data_base_dir/'results'/'simple_correction_model'/'predictions_test_data.csv'
test_results.to_csv(out_file)